In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install rouge_score

In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
MODEL_NAME = "sbtraining2020/email_gemma"

In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
hf_tokenizer = AutoTokenizer.from_pretrained( MODEL_NAME) # "sbtraining2020/esubjectgen_llama31_clean")
hf_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME) #"sbtraining2020/esubjectgen_llama31_clean")

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

...

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

In [6]:
alpaca_prompt = """Below is a instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [7]:
def get_output(email_text):
  #"Generate a summary for the Input section in not more than 50 words"
  inputs = hf_tokenizer(
                        [
                            alpaca_prompt.format(
                                # "Generate a subject for the email body defined in Input section in not more than 50 words", # instruction
                                "Generate a subject line for the following email",
                                email_text, # input
                                "", # output - leave this blank for generation!
                            )
                        ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(hf_tokenizer)
  # result = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


  result = hf_model.generate(    **inputs,
    #streamer = text_streamer,
    max_new_tokens=128,  # Limit length to encourage short outputs
    num_return_sequences=1,  # Generate one sequence
    no_repeat_ngram_size=2,  # Avoid repeating phrases
    early_stopping=True  # Stop when the model thinks it’s done
    )

  decoded = hf_tokenizer.batch_decode(result, skip_special_tokens=True)
  response_text = decoded[0].split("### Response:")[-1].strip().replace('<|end_of_text|>','').replace('<|begin_of_text|>:// ','').replace('Subject: ','').split('\n')[0]
  return response_text[0] if isinstance(response_text, list) else response_text



In [8]:
email_text = """Plove is going to go to Dallas.
We are going to leave next Friday when he  gets done (7ish) and go up for the game.
The game is at 11 in the morning,  so we will come home directly after it.
Plove says he has a friend who has a  place in Dallas that we can crash at if we don't want to pay for a hotel.
Do you want to go?
        """

In [ ]:
#email_text = """The following reports have been waiting for your approval for more than 4 days. Please review. Owner: James W Reitmeyer Report Name: JReitmeyer 10/24/01 Days In Mgr. Queue: 5 """


In [9]:
result = get_output(email_text)

In [10]:
result

'Dallas'